In [1]:
#https://github.com/Microsoft/LightGBM/blob/master/examples/python-guide/simple_example.py
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import random

_a = 0.3
_b = 0.5

f = lambda x: _a * x + _b # ground truth
g = lambda x: f(x) + random.gauss(0, 0.02) # a noisy version of f

n = 50 # number of examples

xs = [random.random() for _ in range(n)] # features
ys = [g(x) for x in xs] # targets

txs = [random.random() for _ in range(n)] # features
tys = [f(x) for x in txs] # we don't get to see this


def l2_loss(ys, ps):
    #print(list(zip(ys, ps.get_label())))
    a = sum(map(lambda t: (t[0] - t[1])**2, zip(ys, ps.get_label())))/len(ys)
    #a = sum(map(lambda y, p: (y - p)**2, zip(ys, ps)))
    return "l2_loss", a, False

def l2_loss_obj(ys, ps):
    g = list(map(lambda t: 2*(t[0] - t[1]), zip(ys, ps.get_label())))
    h = list(map(lambda t: -2, zip(ys, ps.get_label())))
    return g,h
    
# load or create your dataset
df_train = pd.DataFrame(np.concatenate((np.array(xs).reshape(-1,1), np.array(ys).reshape(-1,1)), axis=1))
df_test = pd.DataFrame(np.concatenate((np.array(txs).reshape(-1,1), np.array(tys).reshape(-1,1)), axis=1))



In [2]:
y_train = df_train[1]
y_test = df_test[1]
X_train = df_train.drop(1, axis=1)
X_test = df_test.drop(1, axis=1)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, np.array(y_train))
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [3]:
y_test.shape

(50,)

In [4]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                feval=l2_loss,
                fobj=l2_loss_obj,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
#gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

[1]	valid_0's l1: 0.628344	valid_0's l2: 0.403753	valid_0's l2_loss: 0.403753
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.628344	valid_0's l2: 0.403753	valid_0's l2_loss: 0.403753
[3]	valid_0's l1: 0.628344	valid_0's l2: 0.403753	valid_0's l2_loss: 0.403753
[4]	valid_0's l1: 0.628344	valid_0's l2: 0.403753	valid_0's l2_loss: 0.403753
[5]	valid_0's l1: 0.628344	valid_0's l2: 0.403753	valid_0's l2_loss: 0.403753
[6]	valid_0's l1: 0.628344	valid_0's l2: 0.403753	valid_0's l2_loss: 0.403753
Early stopping, best iteration is:
[1]	valid_0's l1: 0.628344	valid_0's l2: 0.403753	valid_0's l2_loss: 0.403753
Saving model...
Starting predicting...
The rmse of prediction is: 0.635415262855751


In [14]:
from sklearn.preprocessing import OneHotEncoder

# to jest ważne

In [5]:
import time
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

print(__doc__)

# Author: Arthur Mensch <arthur.mensch@m4x.org>
# License: BSD 3 clause

# Turn down for faster convergence
t0 = time.time()
train_samples = 5000

# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

lgb_train = lgb.Dataset(X_train, np.array(y_train))
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


Automatically created module for IPython interactive environment


In [70]:
%%time
params = {
    'boosting_type': 'gbdt',
    'num_class': 10, 
    'objective': "None" ,
    'metric': None,
    'num_leaves': 5,
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                fobj= lgbm_objective,
                feval=lgbm_kaggle_multilog_loss,
#                 fobj=l2_loss_obj,
                early_stopping_rounds=5)

/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)
/home/mf385423/miniconda3/envs/plasticc/lib/p

[1]	valid_0's kaggle_multilog_loss: 39.8371
Training until validation scores don't improve for 5 rounds.


/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)
/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


[2]	valid_0's kaggle_multilog_loss: 39.1439


/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)
/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


...

/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)
/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


[18]	valid_0's kaggle_multilog_loss: 36.9467


/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)
/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


[19]	valid_0's kaggle_multilog_loss: 36.8927
[20]	valid_0's kaggle_multilog_loss: 36.8414
Did not meet early stopping. Best iteration is:
[20]	valid_0's kaggle_multilog_loss: 36.8414
CPU times: user 11min 16s, sys: 1.53 s, total: 11min 17s
Wall time: 21.5 s


/home/mf385423/miniconda3/envs/plasticc/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


In [48]:
@jit
def lgbm_kaggle_multilog_loss(y_pred, y_true ):
    """ Implemented multi log loss from kaggle competition (Light GBM version) """
    #Analised weights for classes
    magic_const = 1.9188
    weight_dict = [1., 2., 1., 1., 1., 1., 1., 2., 1., 1. ]
    y_true = OneHotEncoder(sparse=False, n_values=len(weight_dict)).fit_transform(y_true.get_label().reshape(-1, 1))
    y_pred = y_pred.reshape(-1, len(weight_dict))
    

    log_p = np.log(y_pred)
    N = np.sum(y_true, axis=0)
    weights = np.array(weight_dict)*magic_const
    denominator = np.sum(weights)
    return 'kaggle_multilog_loss', -np.sum(np.sum(np.multiply(y_true, log_p), axis=0)*weights/N/denominator), False

In [46]:
from numba import jit

In [59]:

def lgbm_objective(y_pred, y_true):
    """ Implemented multi log loss objective from kaggle competition (Light GBM version) """
    #Analised weights for classes
    magic_const = 1.9188
    weight_dict = [1., 2., 1., 1., 1., 1., 1., 2., 1., 1. ]
    y_true = OneHotEncoder(sparse=False).fit_transform(y_true.get_label().reshape(-1, 1))
    y_pred = y_pred.reshape(-1, y_true.shape[1])
    #evading log extremes
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)

    #calculating gradient
    inverted_p = 1/y_pred
    N = np.sum(y_true, axis=0)
    weights = np.array(list(weight_dict))*magic_const
    denominator = np.sum(weights)
    gradient = -np.multiply(y_true, inverted_p)/denominator/N*weights
    #-(np.sum(np.multiply(np.multiply(y_true, inverted_p),weights)/N, axis=1))/denominator
    

    #calculating hessian
    #inverted_p2 = -1/inverted_p
    hessian = -np.multiply(gradient,inverted_p)
    #np.sum((np.multiply(np.multiply(y_true, inverted_p2),weights)/N), axis=1)/denominator
    
    return  np.nan_to_num(gradient.flatten()), np.nan_to_num(hessian.flatten()) #np.ones(hessian.flatten().shape)